In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import holidays

In [2]:
train_df = pd.read_parquet("project/bike_counters/data/train.parquet")
train_df = train_df[['counter_name', 'date', 'latitude', 'longitude', 'log_bike_count']]

test_df = pd.read_parquet("project/bike_counters/data/final_test.parquet")

weather_df = pd.read_csv("project/bike_counters/external_data/external_data.csv")
# Drop columns with many nan values
threshold = len(weather_df) * 0.8
weather_df = weather_df.dropna(axis=1, thresh=threshold)
weather_df = weather_df[["date", "t","ff", "pres", "rafper", "u", "vv", "rr1", "rr3", "rr6", "rr12"]]
# Replace negative values in the 'rr1' column with 0
weather_df['rr1'] = weather_df['rr1'].apply(lambda x: max(x, 0))


In [3]:
train_df.head()

,counter_name,date,latitude,longitude,log_bike_count
48321,28 boulevard Diderot E-O,2020-09-01 02:00:00,48.846028,2.375429,0.000000
48324,28 boulevard Diderot E-O,2020-09-01 03:00:00,48.846028,2.375429,0.693147
48327,28 boulevard Diderot E-O,2020-09-01 04:00:00,48.846028,2.375429,0.000000
48330,28 boulevard Diderot E-O,2020-09-01 15:00:00,48.846028,2.375429,1.609438
48333,28 boulevard Diderot E-O,2020-09-01 18:00:00,48.846028,2.375429,2.302585


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 496827 entries, 48321 to 929187
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   counter_name    496827 non-null  category      
 1   date            496827 non-null  datetime64[us]
 2   latitude        496827 non-null  float64       
 3   longitude       496827 non-null  float64       
 4   log_bike_count  496827 non-null  float64       
dtypes: category(1), datetime64[us](1), float64(3)
memory usage: 19.4 MB


In [5]:
weather_df.head()

,date,t,ff,pres,rafper,u,vv,rr1,rr3,rr6,rr12
0,2021-01-01 00:00:00,272.75,1.8,99680,2.5,96,990,0.0,0.0,0.0,0.0
1,2021-01-01 03:00:00,271.25,1.7,99790,2.2,98,210,0.0,0.0,0.0,0.0
2,2021-01-01 06:00:00,271.95,2.6,99820,3.2,98,3660,0.0,0.0,0.0,0.0
3,2021-01-01 09:00:00,272.45,1.7,99970,2.3,97,3500,0.0,0.2,0.2,0.2
4,2021-01-01 12:00:00,276.95,1.0,100000,4.4,82,8000,0.0,0.0,0.2,0.2


In [6]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3322 non-null   object 
 1   t       3322 non-null   float64
 2   ff      3322 non-null   float64
 3   pres    3322 non-null   int64  
 4   rafper  3322 non-null   float64
 5   u       3322 non-null   int64  
 6   vv      3322 non-null   int64  
 7   rr1     3313 non-null   float64
 8   rr3     3316 non-null   float64
 9   rr6     3306 non-null   float64
 10  rr12    3300 non-null   float64
dtypes: float64(7), int64(3), object(1)
memory usage: 285.6+ KB


In [7]:
weather_df.describe()

,t,ff,pres,rafper,u,vv,rr1,rr3,rr6,rr12
count,3322.000000,3322.000000,3322.000000,3322.000000,3322.000000,3322.000000,3313.000000,3316.000000,3306.000000,3300.000000
mean,285.877905,3.654154,100600.930163,7.067128,74.775135,20142.338952,0.086870,0.241556,0.493406,0.994970
std,6.851700,2.000353,913.831886,3.614242,17.022427,10283.556918,0.504275,1.059991,1.711872,2.718507
min,267.650000,0.000000,96190.000000,0.600000,24.000000,120.000000,0.000000,-0.100000,-0.100000,-0.100000
25%,280.950000,2.200000,100130.000000,4.400000,63.000000,12710.000000,0.000000,0.000000,0.000000,0.000000
50%,285.850000,3.400000,100750.000000,6.600000,79.000000,20000.000000,0.000000,0.000000,0.000000,0.000000
75%,290.650000,4.900000,101200.000000,9.200000,89.000000,25000.000000,0.000000,0.000000,0.000000,0.600000
max,307.450000,14.600000,102790.000000,27.900000,100.000000,60000.000000,14.900000,20.100000,23.700000,33.600000


In [8]:
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df.set_index('date', inplace=True)

# Resample to 1-hour intervals and interpolate missing data
weather_df = weather_df.resample('1H').mean().interpolate(method='linear')

# Reset the index to make 'date' a column again
weather_df.reset_index(inplace=True)

# Merge the DataFrames on the 'date' column
merged_df = pd.merge(train_df, weather_df, on='date', how='inner')

# Display the first few rows of the merged DataFrame
merged_df.head()

C:\Users\Magdalena Jankowska\AppData\Local\Temp\ipykernel_21064\798336980.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  weather_df = weather_df.resample('1H').mean().interpolate(method='linear')


,counter_name,date,latitude,longitude,log_bike_count,t,ff,pres,rafper,u,vv,rr1,rr3,rr6,rr12
0,28 boulevard Diderot E-O,2020-09-01 02:00:00,48.846028,2.375429,0.000000,284.55,1.266667,100920.000000,2.033333,85.666667,26666.666667,0.0,0.0,0.0,0.0
1,28 boulevard Diderot E-O,2020-09-01 03:00:00,48.846028,2.375429,0.693147,283.95,1.100000,100900.000000,1.500000,88.000000,25000.000000,0.0,0.0,0.0,0.0
2,28 boulevard Diderot E-O,2020-09-01 04:00:00,48.846028,2.375429,0.000000,284.05,1.333333,100903.333333,2.966667,89.000000,25000.000000,0.0,0.0,0.0,0.0
3,28 boulevard Diderot E-O,2020-09-01 15:00:00,48.846028,2.375429,1.609438,293.65,4.000000,100690.000000,7.500000,41.000000,30000.000000,0.0,0.0,0.0,0.0
4,28 boulevard Diderot E-O,2020-09-01 18:00:00,48.846028,2.375429,2.302585,292.15,3.000000,100700.000000,6.500000,47.000000,30000.000000,0.0,0.0,0.0,0.0


In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496827 entries, 0 to 496826
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   counter_name    496827 non-null  category      
 1   date            496827 non-null  datetime64[us]
 2   latitude        496827 non-null  float64       
 3   longitude       496827 non-null  float64       
 4   log_bike_count  496827 non-null  float64       
 5   t               496827 non-null  float64       
 6   ff              496827 non-null  float64       
 7   pres            496827 non-null  float64       
 8   rafper          496827 non-null  float64       
 9   u               496827 non-null  float64       
 10  vv              496827 non-null  float64       
 11  rr1             496827 non-null  float64       
 12  rr3             496827 non-null  float64       
 13  rr6             496827 non-null  float64       
 14  rr12            496827 non-null  flo

In [10]:
# Merge the DataFrames on the 'date' column
merged_df_test  = pd.merge(test_df, weather_df, on='date', how='inner')

In [ ]:
holidays = holidays.CountryHoliday('France')
def is_holiday(date): # 1: holiday, 0: not holiday
    return 1 if date in holidays else 0

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour
    X['day_of_week'] = X['date'].dt.dayofweek

    X['is_weekend'] = X['day_of_week'].apply(lambda x: 1 if x >= 5 else 0) # 1: weekend, 0: weekday
    X['is_holiday'] = X['date'].apply(is_holiday)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [12]:
def _encode_features(X):
    X = X.copy()
    #X['temp_hour'] = X['t'] * X['hour_sin']
    X['temp_hour'] = X['t'] * X['hour']
    X['weekend_temp'] = X['t'] * X['is_weekend']


    # Create comfort index (simplified version of feels-like temperature)
    X['comfort_index'] = X['t'] - 0.55 * (1 - X['u']/100) * (X['t'] - 14)

    # Rain intensity categories
    X['rain_intensity'] = (X['rr1'] > 0).astype(int) + \
                        (X['rr3'] > 0).astype(int) + \
                        (X['rr6'] > 0).astype(int) + \
                        (X['rr12'] > 0).astype(int)

    # Wind categories
    X['high_wind'] = (X['ff'] > X['ff'].mean() + X['ff'].std()).astype(int)
    return X.drop(columns=["rr3", "rr6", "rr12"])

In [13]:
'''categorical_cols = ['counter_name', 'is_weekend', 'is_holiday']
numerical_cols = ['latitude', 'longitude',  "t","ff", "pres", "rafper", "u", "vv", "rr1"]'''

'categorical_cols = [\'counter_name\', \'is_weekend\', \'is_holiday\']\nnumerical_cols = [\'latitude\', \'longitude\',  "t","ff", "pres", "rafper", "u", "vv", "rr1"]'

In [14]:
# Encode date features
merged_df = _encode_dates(merged_df)
merged_df = _encode_features(merged_df)

# Define feature columns
categorical_columns = ['counter_name', 'is_weekend', 'is_holiday']
numerical_columns = ["latitude", "longitude", "t", "ff", "pres", "rafper", "u", "vv", "rr1", "year", "month", "day", "weekday", "hour", 'day_of_week', 'temp_hour', 'weekend_temp', 'comfort_index', 'rain_intensity', 'high_wind']
target_column = "log_bike_count"

# Split data into features and target
X = merged_df[categorical_columns + numerical_columns]
y = merged_df[target_column]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing for numerical and categorical columns
numerical_preprocessor = StandardScaler()
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_preprocessor, numerical_columns),
        ("cat", categorical_preprocessor, categorical_columns),
    ]
)

# Define the XGBoost model
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# Create the pipeline
pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", xgb_model)])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 0.43678635016256384


In [15]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    "model__n_estimators": [250, 300, 350],
    "model__max_depth": [ 8, 9, 10],
    "model__learning_rate": [0.25, 0.3, 0.4]
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation
    scoring="neg_mean_squared_error",
    verbose=1,
    n_jobs=-1,  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Evaluate the model with the best parameters
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
print(f"Best Model RMSE: {rmse_best}")


Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'model__learning_rate': 0.25, 'model__max_depth': 9, 'model__n_estimators': 350}
Best Model RMSE: 0.35469338969074843


In [16]:
merged_df_test = _encode_dates(merged_df_test)
merged_df_test = _encode_features(merged_df_test)
X_pred = merged_df_test
y_pred = best_model.predict(X_pred)
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission_new.csv", index=False)